# **SeqOne** - *Analyse et Machine Learning*

Sommaire :
1. Analyse exploratoire
2. Pre-processing
3. Machine Learning
4. Propositions d'améliorations

## 1. Analyse exploratoire

### Import des librairies

In [ ]:
    # data analysis libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

    # machine learning libraries
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import cross_validate, cross_val_score
import sklearn.metrics as metrics
from sklearn.metrics import ConfusionMatrixDisplay

    # loading the dataset

df = pd.read_csv("dataframe.tsv", sep ="\t")
df.head()

### a) caractéristiques du jeu de données

#### *- générales*

In [ ]:
print(f"Dimensions : {df.shape}.\n")
df.info()

___
Le jeu de données est constitué de 438 lignes et 40 colonnes, dont 2 colonnes index (build_variant_id et contigName), 1 colonne *target* (class) et 37 colonnes *features*.

Les données sont numériques (booléennes, continues ou catégorielles), hormis la colonne d'index 0.
___

#### *- statistiques*

In [ ]:
numerical_columns_stats=df.describe().transpose()
print(numerical_columns_stats)

___
Observations :
+ Aucune variable n’est négative. 
+ La plupart d’entre elles ne dépassent pas la valeur 1. 
+ Certaines ne dépassent pas un certain entier, aux alentours de 7 ou 8.
+ Rarement, des variables ont un plus grand écart-type qui dépasse la centaine.
+ Un nombre non-négligeable de ces variables ont seulement deux valeurs : 0 ou 1, elles sont certainement de type booléen.
___

### b) valeurs manquantes (NaN)

#### *- colonnes*

In [ ]:
df_na = df.isna()
for column_name in df_na.columns[3:]:
    column = df_na[column_name]
    count = int(((column == True).sum()/438)*100)
    df_na = df_na.rename(columns={column_name:f"{column_name} ({count}%)"})

plt.suptitle('Valeurs NaN (en blanc)', fontweight='bold' )
sns.heatmap(df_na, xticklabels=True, yticklabels=False, cbar=False, linewidths=0)
plt.xticks(rotation = 90, fontsize=7)
plt.xlabel("Columns (NaN %)")
plt.ylabel("Rows")
plt.show()

___
+ Sur la base de la distribution des valeurs manquantes, on peut supposer des relations entre différentes colonnes :
    + entre spa et sprf
    + EXON et INTRON
    + gene_cluster, lof_gene, missense_gene
    + peut-être gfq et sft
___

#### *- lignes*

Un cluster de valeurs manquantes est observé dans Spyder :
![https://ibb.co/Kw1zHyR](https://i.ibb.co/6BdJ5sV/lines-missing.png)

In [ ]:
print(f"Ce cluster est constitué de {df['gene_cluster'].isna().sum()} valeurs manquantes.")

### c) doublons

In [ ]:
print(f"Il y a {df.duplicated().sum()} doublon(s).")

### d) valeurs égales à zéro

In [ ]:
df_zero = (df == 0)
for column_name in df_zero.columns[3:]:
    column = df_zero[column_name]
    count = int(((column == True).sum()/438)*100)
    df_zero = df_zero.rename(columns={column_name:f"{column_name} ({count}%)"})

plt.suptitle('Valeurs égale à 0 (en blanc)', fontweight='bold' )
sns.heatmap(df_zero, xticklabels=True, yticklabels=False, cbar=False, linewidths=0)
plt.suptitle('Valeurs égales à 0 (en blanc)', fontweight='bold' )
plt.xticks(rotation = 90, fontsize=7)
plt.xlabel("Columns (zero values %)")
plt.ylabel("Rows")
plt.show()

___
+ De nombreuses valeurs égales à zéro sont observées parmi les différentes colonnes.
+ Les colonnes d’index 3, 6, 7, 8, 9, 10, 15, 30, 32, 38 et 39 ont à plus de 90% la valeur zéro
___

### e) valeurs uniques

In [ ]:
for column_name in df.columns[2:]:
    column = df[column_name]
    unique = column.unique()
    nunique = column.nunique()
    print(f"\n{nunique} unique values in '{column_name}':\n{unique}")

___
+ 'stop_retained_variant' n'a qu'une seule valeur unique.
+ Parmi les colonnes de type int64, les colonnes 17, 19, 21, 23, 38 et 39 n’ont que deux valeurs. Il semblerait qu’elles soient des variables booléennes.
___

### f) type de données

In [ ]:
ser1=pd.Series(df.dtypes)
ser2=df.nunique()
df_dtypes= pd.concat([ser1, ser2], axis="columns").rename(columns={0:"dtype", 1:"nunique"})
print(df_dtypes)

___
On peut en déduire que, malgré le fait que la plupart des variables sont de type float64, elles ont un nombre limité de valeurs possible. Cela signifie probablement que ces données sont davantage catégorielles ou booléennes que continues.
___

### g) distribution de chaque colonne

In [ ]:
# distribution de chaque colonne sous forme de barplot [1/4]
# étant donné que la plupart des colonnes sont catégorielles

for x in range(2,10):
    groupby = df.groupby(df.columns[x])[df.columns[x]].count()
    
    plt.figure(x)
    sns.barplot(x=groupby.index, y=groupby)
    plt.ylim(0, 438)
    plt.xlabel("Unique Values")
    plt.ylabel("Total Count")
    plt.suptitle(df.columns[x], fontweight='bold')

In [ ]:
# [2/4]

for x in range(10,20):
    groupby = df.groupby(df.columns[x])[df.columns[x]].count()
    
    plt.figure(x)
    sns.barplot(x=groupby.index, y=groupby)
    plt.ylim(0, 438)
    plt.xlabel("Unique Values")
    plt.ylabel("Total Count")
    plt.suptitle(df.columns[x], fontweight='bold')

In [ ]:
# [3/4]

for x in range(20,30):
    groupby = df.groupby(df.columns[x])[df.columns[x]].count()
    
    plt.figure(x)
    sns.barplot(x=groupby.index, y=groupby)
    plt.ylim(0, 438)
    plt.xlabel("Unique Values")
    plt.ylabel("Total Count")
    plt.suptitle(df.columns[x], fontweight='bold')

In [ ]:
# [4/4]

for x in range(30,len(df.columns)):
    groupby = df.groupby(df.columns[x])[df.columns[x]].count()
    
    plt.figure(x)
    sns.barplot(x=groupby.index, y=groupby)
    plt.ylim(0, 438)
    plt.xlabel("Unique Values")
    plt.ylabel("Total Count")
    plt.suptitle(df.columns[x], fontweight='bold')

___
Maintenant que le jeu de données est connu dans ses grandes lignes, on peut passer à l'étape de pre-processing, afin de le rendre exploitable par les algorithmes de machine learning.
___

## 2. Pre-processing

### a) Suppression des colonnes inutiles

In [ ]:
    # les index
print(f'Index columns:')
df_ml = df.drop(columns=['build_variant_id', 'contigName'])
print(f'\tDropped column: build_variant_id.')
print(f'\tDropped column: contigName.')

    # les colonnes ayant plus de 50% de NaN
print('\n>50% NaN values columns:')
for x in df_ml :
        if df_ml[x].isnull().sum() > (len(df. index)/2):
            df_ml = df_ml.drop(columns=[x])
            print(f'\tDropped column: {x}.')

    # les colonnes uniformes
print('\nUniform columns:')
for x in df_ml :
        if df_ml[x].nunique()==1:
            df_ml=df_ml.drop(columns=[x])
            print(f'\tDropped column: {x}.')

___
Les colonnes qui ont plus de 50% de données manquantes ont été supprimées, ainsi que les colonnes uniformes ou non-numériques.
___

### b) suppression des lignes inutiles

___
Précédemment, un cluster de lignes avec plusieurs valeurs manquantes a été identifié. Mais elles ne seront pas supprimées car le nombre de lignes du jeu de données est déjà limité.
___

### c) traitement des valeurs manquantes

In [ ]:
    # colonnes booléennes

bool_cols=[]

for x in df_ml :
        if df_ml[x].nunique() == 2:
            for y in df_ml[x].unique():
                if y == (0 or 1):
                    bool_cols.append(x)

df_ml[bool_cols]=df_ml[bool_cols].fillna(0)
print('Boolean columns filled with 0:\n', bool_cols)

    # colonnes numériques continues

cont_cols = []

for x in df_ml:
        if df_ml[x].nunique() > 30:
            cont_cols.append(x)

df_ml[cont_cols]=df_ml[cont_cols].fillna(df[cont_cols].median())
print('\nContinuous columns filled with median:\n', cont_cols)

    # colonnes catégoriques

cat_cols = []

for x in df_ml:
        if df_ml[x].nunique() < 30 and df_ml[x].nunique() > 2:
            cat_cols.append(x)
        
df_ml[cat_cols]=df_ml[cat_cols].fillna(df_ml[cat_cols].median())
print('\nCategorical columns filled with median:\n', cat_cols)
print('Number of columns after pre-processing:', len(bool_cols + cont_cols + cat_cols))

___
Les valeurs manquantes ont été remplacées de la manière suivante :
+ celles des colonnes booléennes par 0 ;
+ celles des colonnes continues par la médiane ;
+ celles des colonnes catégoriques par la valeur médiane.
___

___
Le jeu de données a été allégé des colonnes jugées inutiles et n'en compte plus que 32, dont :
+ 6 colonnes de type booléen ;
+ 8 colonnes de type continu ;
+ 18 colonnes de type catégoriel.

Il est constitué entièrement de valeurs numériques. Il est donc exploitable pour le Machine Learning.
___

## 3. Machine Learning

## a) méthodologie

Nous avons affaire à un problème de classification *multi-class*.

La colonne *class* est la variable à prédire (target), les autres sont les *features*. On rappelle que la colonne *class* peut prendre ces valeurs :

In [ ]:
df['class'].unique()

Et que sa distribution est la suivante :

In [ ]:
df['class'].groupby(df['class']).count()

La classe 3 est la plus représentée, la classe 4 la moins.

___
Etant un problème de classification *multi-class*, le modèle *Random Forest* a été choisi.

On va d'abord faire un premier modèle de *Random Forest* avec les valeurs par défaut nommé *Default Model*. 

Puis, après avoir fait des mesures de performances, paramétrer un deuxième modèle appelé *Optimized Model* afin d'améliorer les performances du premier.

Enfin, on comparera les deux modèles pour voir lequel est le plus performant.

### b) split train-test

In [ ]:
y = df_ml['class']
X = df_ml.drop(columns=['class'])
rd_state = 2

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=rd_state)

### c) choix du modèle

In [ ]:
model = RandomForestClassifier(random_state=rd_state)
model.fit(X_train, y_train)

### d) mesure des performances

#### *- Accuracy*

In [ ]:
y_pred = model.predict(X_val)
accuracy = metrics.accuracy_score(y_val, y_pred)
print("Accuracy: ", accuracy)

#### *- Confusion Matrix*

In [ ]:
confusion = metrics.confusion_matrix(y_val, y_pred)
# print(f"Confusion matrix:\n{confusion}")
conf_norm = confusion / confusion.astype(float).sum(axis=1)
# print(f"Normalized confusion matrix:\n{conf_norm}")

ConfusionMatrixDisplay(confusion).plot()
plt.suptitle("Default Model", fontweight='bold')
plt.title(f'Confusion Matrix (acc: {round(accuracy, 3)})', style='italic')

ConfusionMatrixDisplay(conf_norm).plot()
plt.suptitle("Default Model", fontweight='bold')
plt.title(f'Normalized Confusion Matrix (acc: {round(accuracy, 3)})', style='italic')
plt.show()

Notons que l'apprentissage du modèle a modifié le nom des différentes classes respectivement :
    
    (1, 2, 3, 4, 5) --> (0, 1, 2, 3, 4)
    
On peut observer que :
+ Les trois premières classes sont relativement bien prédites par le modèle (leur meilleure représentation au sein du dataset l'explique certainement) ;
+ au contraire des deux dernières (par ailleurs, les moins bien représentées).

___
Remarquons que le score de précision est calculé de manière générale : le poids de chaque classe dans ce score est proportionnel à leur distribution dans le jeu de données.

Si le poids de chaque classe est ramené à 1, on obtient un score moins élevé :

In [ ]:
conf_sum = 0
for x in range(5):
    conf_sum = conf_sum + conf_norm[x, x]
conf_mean = conf_sum/(x+1)
print(f"Précision du modèle calculée avec des poids égaux : {round(conf_mean, 3)}.")

#### *- Cross Validation*

In [ ]:
cv_results = cross_validate(model, X, y, cv=10)
sorted(cv_results.keys())
print(cv_results['test_score'])

scores = cross_validate(model, X, y, cv=4,
                        scoring=('r2', 'accuracy'),
                        return_train_score=True)

print(f"\nMoyenne du CV: {scores['test_accuracy'].mean()}.")

### e) mesure de chaque paramètre

#### *- random state*

On fait varier le random state du splittage pour estimer la tendance à l'overfitting du modèle.

In [ ]:
y = df_ml['class']
X = df_ml.drop(columns=['class'])
acc_list = []

for x in range(20):
    X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=None)
    
    model = RandomForestClassifier(random_state=None)
    model.fit(X_train, y_train)
    
    # mesure des performances
    
        # accuracy
    
    y_pred = model.predict(X_val)
    accuracy = metrics.accuracy_score(y_val, y_pred)
    # print("Accuracy: ", accuracy)
    acc_list.append(accuracy)
    
        # plot

plt.figure(1)
plt.ylim(0.4,1)
plt.xlim(0,20)
plt.suptitle("Default Model", fontweight='bold')
plt.title(f"Accuracy Through {len(acc_list)} Random Batches", style="italic")
plt.xlabel("Batch n°")
plt.ylabel("Accuracy Score")
plt.text(12, 0.55, f"Accuracy mean: {round(sum(acc_list)/len(acc_list), 2)}", style="italic")
plt.plot(acc_list, marker="o", linewidth=0.5)
plt.grid(True)
plt.show()

La précision du modèle varie entre 0.65 et 0.8 selon que le dataset est shufflisé avant d'être splitté. Cette variation n'est pas négligeable.

### *- n_estimators*

In [ ]:
n_estimators = np.arange(1,150, step=2)
train_estimator_results = []

for estimator in n_estimators:
    X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=rd_state)
    rf = RandomForestClassifier(n_estimators=estimator, random_state=rd_state, n_jobs=-1)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)
    accuracy = metrics.accuracy_score(y_val, y_pred)
    train_estimator_results.append(accuracy)
    
plt.figure(2)
plt.plot(n_estimators, train_estimator_results)
plt.xlabel('Number of trees')
plt.ylabel('Accuracy')
plt.ylim(0.4, 1)
plt.grid(True)
plt.suptitle("Default Model", fontweight='bold')
plt.title("Accuracy vs. Number of Trees", style="italic")
plt.show()

Le nombre idéal d'arbres atteint un plafond aux alentours de 70 arbres.

#### *- max_depth*

In [ ]:
max_depths = np.arange(1, 30, step=1)
train_depth_results = []

for depth in max_depths:
    X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=rd_state)
    model = RandomForestClassifier(n_estimators=70, max_depth=depth, random_state=rd_state)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    accuracy = metrics.accuracy_score(y_val, y_pred)
    train_depth_results.append(accuracy)


plt.figure(3)
plt.plot(max_depths, train_depth_results)
plt.xlabel('Depth')
plt.ylabel('Accuracy')
plt.ylim(0.4, 1)
plt.grid(True)
plt.suptitle("Default Model", fontweight='bold')
plt.title("Accuracy against Depth of the trees", style="italic")
plt.show()

La précision du modèle atteint un plafond lorsque la profondeur des arbres atteint 15.

#### *- train test split*

In [ ]:
y = df_ml['class']
X = df_ml.drop(columns=['class'])
train_test_size_results = []

test_sizes = np.arange(0.01, 1, 0.02)

for test_size in test_sizes:
    X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=rd_state, test_size=test_size)
    model = RandomForestClassifier(max_depth=15, random_state=rd_state, n_estimators=70)
    model.fit(X_train, y_train)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    accuracy = metrics.accuracy_score(y_val, y_pred)
    train_test_size_results.append(accuracy)
    
plt.figure(4)
plt.plot(test_sizes, train_test_size_results)
plt.xlabel('Test Size Ratio')
plt.ylabel('Accuracy')
plt.ylim(0.4, 1)
plt.grid(True)
plt.suptitle("Default Model", fontweight='bold')
plt.title("Accuracy against Test/train ratio", style="italic")
plt.show()

Consacrer environ 30% des données au jeu de test est le ratio qui assure la meilleure précision du modèle.

### f) optimisation des paramètres du modèle

Maintenant que nous avons les valeurs optimales de chaque paramètre du modèle, nous pouvons les remplacer pour générer le nouveau modèle *Optimized Model*, puis nous le comparerons avec le *Default Model*.

In [ ]:
y = df_ml['class']
X = df_ml.drop(columns=['class'])
optim_acc_list = []

#### *- mesure des performances*

In [ ]:
for x in range(20):
    X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=None, test_size=0.3)
    
    optimized_model = RandomForestClassifier(random_state=rd_state, n_estimators = 70, max_depth=15)
    optimized_model.fit(X_train, y_train)    

    y_pred = optimized_model.predict(X_val)
    accuracy = metrics.accuracy_score(y_val, y_pred)
    # print("Accuracy: ", accuracy)
    optim_acc_list.append(accuracy)

plt.figure(1)
plt.ylim(0.4,1)
plt.xlim(0,20)
plt.suptitle("Optimized Model", fontweight='bold')
plt.title(f"Accuracy Through {len(acc_list)} Random Batches", style="italic")
plt.xlabel("Batch n°")
plt.ylabel("Accuracy Score")
plt.grid(True)
plt.text(7.3, 0.47, f"Optimized Accuracy Mean: {round(sum(optim_acc_list)/len(optim_acc_list), 2)}\n\nDefault Accuracy Mean: {round(sum(acc_list)/len(acc_list), 2)}", style="italic")
plt.plot(optim_acc_list, marker="o", linewidth=0.5)
plt.show()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=rd_state)

optimized_model = RandomForestClassifier(random_state=rd_state, n_estimators = 35, max_depth=15)
optimized_model.fit(X_train, y_train)
y_pred = optimized_model.predict(X_val)
accuracy = metrics.accuracy_score(y_val, y_pred)

In [ ]:
confusion = metrics.confusion_matrix(y_val, y_pred)
# print(f"Confusion matrix:\n{confusion}")
conf_norm = confusion / confusion.astype(float).sum(axis=1)
#print(f"Normalized confusion matrix:\n{conf_norm}")

ConfusionMatrixDisplay(confusion).plot()
plt.suptitle("Optimized Model", fontweight='bold')
plt.title(f'Confusion Matrix (acc: {round(accuracy, 3)})', style="italic")
plt.show()

ConfusionMatrixDisplay(conf_norm).plot()
plt.suptitle("Optimized Model", fontweight='bold')
plt.title(f'Normalized Confusion Matrix (acc: {round(accuracy, 3)})', style="italic")
plt.show()

In [ ]:
conf_sum = 0
for x in range(5):
    conf_sum = conf_sum + conf_norm[x, x]
conf_mean = conf_sum/(x+1)
print(f"Précision du modèle calculée avec des poids égaux : {round(conf_mean, 3)}.")

### g) conclusion

Premièrement, la précision de l'*Optimized Model* (0.69) n'est pas plus élevée que celle du *Default Model* (0.70). L'*Optimized Model* ne dépasse pas le plafond de 0.7 de précision déjà atteint par le *Default Model*. Cela peut être expliqué par le fait que les paramètres par défaut du modèle étaient très proches des paramètres optimaux.

En revanche, lorsque le poids de chaque classe est égalisé, l'*Optimized Model* obtient un meilleur score de précision que celui du *Default Model*, 0.64 contre 0.58. Cela indique que les prédictions du modèle sont mieux réparties. Il répond donc mieux à l'algorithme multiclasse recherché et est plus apte à la prédiction de nouvelles données, tandis que le *Default Model* était compétent seulement pour les classes les mieux représentées.

Deuxièmement, la précision atteinte par le modèle est à prendre avec précaution. Le score est étonnament élevée (~0.7) malgré le nombre restreint de données. Le risque d'overfitting de l'algorithme ne doit pas être écarté.

De plus, malgré le fait que le test de Cross Validation et les shuffling effectués au split (*random_state=None*) indiquent des valeurs qui varient de +-0.1 autour de 0.7, j'ai mesuré des variations plus importantes dans les paramètres en faisant varier le random_state du modèle. Un plus grand jeu de données nous permettrait de savoir si la relative stabilité du modèle est réelle ou seulement apparente.

## 4. Propositions d'améliorations

### a) au niveau du pre-processing

+ essayer le *one-hot encoding* pour certaines variables catégoriques ;
+ l'ordre des bases ACGT de la colonne *build_variant_id* sont peut-être de l'information utile à extraire ;
+ être plus restrictif quant au choix des *features* pour diminuer le bruit afin de faciliter la tâche de l'algorithme ;
+ chercher davantage de relations entre les features et les quantifier pour l'algorithme (*mutual information* par exemple).

### b) au niveau du modèle

+ essayer d'autres modèles : combiner plusieurs SGD avec une stratégie *One-vs-All* pour chaque classe. Cela permettrait d'avoir plus de contrôle sur le comportement de l'algorithme, et d'effectuer un travail de précision pour éviter la confusion entre deux classes précises ;
+ utiliser d'autres mesures : impureté Gini, F1 score, ...

### c) au niveau du jeu de données

+ un nombre de données plus conséquent ;
+ une meilleure connaissance-métier des différentes colonnes ;
+ vérifier l'état des capteurs pour les colonnes majoritairement constituées de valeurs NaN.